In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from time import perf_counter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

In [2]:
# Use 5% of the dataset for testing various models
# Split into training and test datasets
train_df, test_df = train_test_split(df.sample(frac = 0.05), test_size=0.2, random_state=0)

NameError: name 'df' is not defined

In [3]:
def create_gen():
    # Load the Images with a generator and Data Augmentation
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.1
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='training',
#         rotation_range=30, # Uncomment to use data augmentation
#         zoom_range=0.15,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.15,
#         horizontal_flip=True,
#         fill_mode="nearest"
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='validation',
#         rotation_range=30, # Uncomment to use data augmentation
#         zoom_range=0.15,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.15,
#         horizontal_flip=True,
#         fill_mode="nearest"
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    
    return train_generator,test_generator,train_images,val_images,test_images

In [4]:
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape': (224, 224, 3),
                'include_top': False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    print(model)
    pretrained_model.trainable = False
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(29, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [5]:
# Dictionary with the models
models = {
#     "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
#     "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
#     "DenseNet169": {"model":tf.keras.applications.DenseNet169, "perf":0},
#     "DenseNet201": {"model":tf.keras.applications.DenseNet201, "perf":0},
#     "EfficientNetB0": {"model":tf.keras.applications.EfficientNetB0, "perf":0},
#     "EfficientNetB1": {"model":tf.keras.applications.EfficientNetB1, "perf":0},
#     "EfficientNetB2": {"model":tf.keras.applications.EfficientNetB2, "perf":0},
#     "EfficientNetB3": {"model":tf.keras.applications.EfficientNetB3, "perf":0},
#     "EfficientNetB4": {"model":tf.keras.applications.EfficientNetB4, "perf":0},
#     "EfficientNetB5": {"model":tf.keras.applications.EfficientNetB4, "perf":0},
#     "EfficientNetB6": {"model":tf.keras.applications.EfficientNetB4, "perf":0},
#     "EfficientNetB7": {"model":tf.keras.applications.EfficientNetB4, "perf":0},
#     "InceptionResNetV2": {"model":tf.keras.applications.InceptionResNetV2, "perf":0},
#     "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "MobileNet": {"model":tf.keras.applications.MobileNet, "perf":0},
#     "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
#     "MobileNetV3Large": {"model":tf.keras.applications.MobileNetV3Large, "perf":0},
#     "MobileNetV3Small": {"model":tf.keras.applications.MobileNetV3Small, "perf":0},
#     "NASNetMobile": {"model":tf.keras.applications.NASNetMobile, "perf":0},
#     "ResNet101": {"model":tf.keras.applications.ResNet101, "perf":0},
#     "ResNet101V2": {"model":tf.keras.applications.ResNet101V2, "perf":0},
#     "ResNet152": {"model":tf.keras.applications.ResNet152, "perf":0},
#     "ResNet152V2": {"model":tf.keras.applications.ResNet152V2, "perf":0},
#     "ResNet50": {"model":tf.keras.applications.ResNet50, "perf":0},
#     "ResNet50V2": {"model":tf.keras.applications.ResNet50V2, "perf":0},
#     "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
#     "VGG19": {"model":tf.keras.applications.VGG19, "perf":0},
#     "Xception": {"model":tf.keras.applications.Xception, "perf":0}
}

# Create the generators
# train_generator, test_generator,train_images,val_images,test_images=create_gen()
print('\n')

# Fit the models
for name, model in models.items():
    
    # Get the model
    m = get_model(model['model'])
    models[name]['model'] = m
    
#     start = perf_counter()
    
#     # Fit the model
#     history = m.fit(train_images,validation_data=val_images,epochs=1,verbose=0)
    
#     # Sav the duration and the val_accuracy
#     duration = perf_counter() - start
#     duration = round(duration,2)
#     models[name]['perf'] = duration
#     print(f"{name:20} trained in {duration} sec")
    
#     val_acc = history.history['val_accuracy']
#     models[name]['val_acc'] = [round(v,4) for v in val_acc]



<function MobileNet at 0x13fb88ee0>


In [6]:
models_result = []

for name, v in models.items():
    models_result.append([ name, models[name], 
                          models[name]['perf']])
    
df_results = pd.DataFrame(models_result, 
                          columns = ['model','val_accuracy','Training time (sec)'])

In [7]:
checkpoint_path = "cp.ckpt"
checkpoint_path = "/Users/aringuyen/Desktop/cp.ckpt"

In [8]:
best_model = df_results.iloc[0]
model = get_model(eval("tf.keras.applications."+ best_model[0]) )
model.load_weights(checkpoint_path)
model

<function MobileNet at 0x13fb88ee0>


In [9]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (224, 224, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [57]:
# Predict the label of the test_images
# img  = load('/Users/aringuyen/Desktop/PROJECTS/Game-SignLanguage/RealTimeObjectDetection/Tensorflow/workspace/images/collectedimages/y/Y1003.jpeg')
# img = load('/Users/aringuyen/Desktop/PROJECTS/Game-SignLanguage/RealTimeObjectDetection/Tensorflow/workspace/images/collectedimages/z/Z1.jpeg') 
# img = load('/Users/aringuyen/Desktop/PROJECTS/Game-SignLanguage/RealTimeObjectDetection/Tensorflow/workspace/images/train/y.24fcec37-9fa9-11eb-add5-8c8caa784d19.jpg')
# img = load('/Users/aringuyen/Desktop/PROJECTS/Game-SignLanguage/RealTimeObjectDetection/Tensorflow/workspace/images/collectedimages/w/W10.jpeg')
img = load('/Users/aringuyen/Desktop/PROJECTS/Game-SignLanguage/RealTimeObjectDetection/Tensorflow/workspace/images/collectedimages/b/b.1348e65c-a01a-11eb-ab8a-f01898155646.jpg')
pred = model.predict(img)
# pred = np.argmax(pred,axis=1)



In [58]:
pred

array([[1.19831304e-06, 5.57351947e-01, 5.97996332e-05, 2.37667191e-05,
        3.71937722e-01, 1.77541333e-05, 1.75256236e-08, 1.31797606e-06,
        2.11181739e-08, 1.06132152e-08, 3.66524855e-09, 6.79350080e-07,
        3.13861779e-06, 2.78311660e-07, 1.75556593e-07, 4.86023111e-10,
        9.85817294e-09, 4.20245897e-06, 7.66823746e-07, 8.76679508e-07,
        4.11077635e-03, 3.56818491e-04, 6.60606474e-02, 6.41705401e-06,
        1.86936044e-09, 3.93128175e-07, 6.11273717e-05, 1.07260846e-07,
        4.96861752e-08]], dtype=float32)

In [59]:
labels = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space'}
labels

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z',
 26: 'del',
 27: 'nothing',
 28: 'space'}

In [60]:
result = labels[pred.argmax(axis=1)[0]]

In [61]:
result

'B'

In [62]:
pred.argmax(axis=1)

array([1])